# Elastic Net 弹性网络

---

## 📚 学习目标

通过本教程，你将学会：
- 理解Elastic Net的原理（L1+L2正则化）
- 掌握l1_ratio参数的作用
- 学会选择Ridge、Lasso和Elastic Net
- 使用交叉验证选择最佳参数

## 📋 前置知识

- 岭回归 (Ridge)
- Lasso回归
- 正则化概念

## ⏱️ 预计时间

20-30分钟

---

## 📖 第1部分：理论背景

### 什么是Elastic Net？

**Elastic Net（弹性网络）** 结合了Ridge（L2）和Lasso（L1）正则化的优点，是两者的折中方案。

### 数学模型

$$J(\theta) = \frac{1}{2m}||X\theta - y||^2 + r\alpha\sum_{j=1}^{n}|\theta_j| + \frac{1-r}{2}\alpha\sum_{j=1}^{n}\theta_j^2$$

其中：
- $\alpha$ 是总体正则化强度
- $r$ 是L1比例（`l1_ratio`），范围[0, 1]
  - r = 0：纯Ridge回归
  - r = 1：纯Lasso回归
  - 0 < r < 1：混合正则化

### 为什么需要Elastic Net？

| 问题 | Lasso的局限 | Elastic Net的解决 |
|------|------------|------------------|
| 高度相关特征 | 只选择一个 | 倾向于一起选择 |
| n < p情况 | 最多选n个特征 | 可选更多特征 |
| 稳定性 | 特征选择不稳定 | 更稳定 |

> 💡 **关键洞察**: Elastic Net = Lasso的稀疏性 + Ridge的稳定性

---

## 💻 第2部分：代码实现

### 步骤1: 导入库和配置

In [ ]:
# ============================================================
# 导入必要的库
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet, ElasticNetCV, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# ============================================================
# 配置
# ============================================================

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 配置完成")

### 步骤2: 生成相关特征数据

In [ ]:
# ============================================================
# 生成具有相关特征的数据
# ============================================================

n_samples = 200
n_features = 20

# 生成基础特征
X_base = np.random.randn(n_samples, 5)

# 创建相关特征：每组3-4个高度相关
X = np.zeros((n_samples, n_features))
X[:, 0:5] = X_base
X[:, 5:10] = X_base + 0.1 * np.random.randn(n_samples, 5)  # 与前5个高度相关
X[:, 10:15] = np.random.randn(n_samples, 5)  # 独立噪声特征
X[:, 15:20] = X_base[:, :5] * 0.5 + 0.2 * np.random.randn(n_samples, 5)  # 中度相关

# 真实系数：只有前5个基础特征重要
true_coefs = np.zeros(n_features)
true_coefs[:5] = [3.0, -2.0, 1.5, -1.0, 0.5]

# 生成目标
y = X @ true_coefs + 0.5 * np.random.randn(n_samples)

# 划分数据
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_SEED
)

# 标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✓ 数据生成完成")
print(f"  特征数: {n_features}")
print(f"  真实重要特征: 0-4")
print(f"  相关特征组: 5-9与0-4高度相关, 15-19中度相关")

### 步骤3: 基础Elastic Net

In [ ]:
# ============================================================
# 基础Elastic Net
# ============================================================

# 创建Elastic Net模型
# alpha: 正则化强度
# l1_ratio: L1占比 (0=Ridge, 1=Lasso, 0.5=均衡)
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000)

# 训练
elastic_net.fit(X_train_scaled, y_train)

# 预测和评估
y_pred = elastic_net.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Elastic Net结果 (α=0.1, l1_ratio=0.5):")
print(f"  测试MSE: {mse:.4f}")
print(f"  测试R²: {r2:.4f}")
print(f"  非零系数: {np.sum(np.abs(elastic_net.coef_) > 1e-10)}")

---

## 🔬 第3部分：比较Ridge、Lasso和Elastic Net

In [ ]:
# ============================================================
# 三种模型对比
# ============================================================

# 创建模型
ridge = Ridge(alpha=0.1)
lasso = Lasso(alpha=0.1, max_iter=10000)
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000)

models = {'Ridge': ridge, 'Lasso': lasso, 'Elastic Net': elastic}

# 训练和评估
results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    results[name] = {
        'mse': mean_squared_error(y_test, y_pred),
        'r2': r2_score(y_test, y_pred),
        'n_nonzero': np.sum(np.abs(model.coef_) > 1e-10),
        'coefs': model.coef_
    }

# 可视化系数
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 真实系数
axes[0, 0].bar(range(n_features), true_coefs, color='green', alpha=0.7)
axes[0, 0].set_title('真实系数', fontweight='bold')
axes[0, 0].set_xlabel('特征索引')
axes[0, 0].axhline(y=0, color='black', linestyle='--', linewidth=0.5)
axes[0, 0].grid(True, alpha=0.3)

colors = {'Ridge': 'blue', 'Lasso': 'orange', 'Elastic Net': 'red'}
for ax, (name, res) in zip([axes[0,1], axes[1,0], axes[1,1]], results.items()):
    ax.bar(range(n_features), res['coefs'], color=colors[name], alpha=0.7)
    ax.set_title(f"{name}\nMSE: {res['mse']:.4f}, 非零: {res['n_nonzero']}", fontweight='bold')
    ax.set_xlabel('特征索引')
    ax.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印结果表格
print("\n" + "="*50)
print("模型性能对比")
print("="*50)
print(f"{'模型':<15} {'MSE':>10} {'R²':>10} {'非零系数':>10}")
print("-"*50)
for name, res in results.items():
    print(f"{name:<15} {res['mse']:>10.4f} {res['r2']:>10.4f} {res['n_nonzero']:>10}")

### l1_ratio对特征选择的影响

In [ ]:
# ============================================================
# 不同l1_ratio的影响
# ============================================================

l1_ratios = [0.1, 0.3, 0.5, 0.7, 0.9]
alpha_fixed = 0.1

fig, axes = plt.subplots(1, 5, figsize=(16, 3))

for ax, l1_ratio in zip(axes, l1_ratios):
    model = ElasticNet(alpha=alpha_fixed, l1_ratio=l1_ratio, max_iter=10000)
    model.fit(X_train_scaled, y_train)
    
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    n_nonzero = np.sum(np.abs(model.coef_) > 1e-10)
    
    ax.bar(range(n_features), model.coef_, alpha=0.7)
    ax.set_title(f'l1_ratio={l1_ratio}\nMSE={mse:.3f}, 非零={n_nonzero}', fontsize=10)
    ax.set_xlabel('特征')
    ax.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('l1_ratio对特征选择的影响 (α=0.1)', y=1.02, fontsize=12, fontweight='bold')
plt.show()

print("\n观察结论:")
print("  - l1_ratio小：更像Ridge，保留更多非零系数")
print("  - l1_ratio大：更像Lasso，产生更稀疏的解")

---

## 📊 第4部分：交叉验证选择参数

In [ ]:
# ============================================================
# 使用ElasticNetCV自动选择最佳参数
# ============================================================

# ElasticNetCV同时优化alpha和l1_ratio
elastic_cv = ElasticNetCV(
    l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99],  # 候选l1_ratio
    alphas=np.logspace(-4, 1, 50),                    # 候选alpha
    cv=5,                                             # 5折交叉验证
    max_iter=10000,
    random_state=RANDOM_SEED
)

elastic_cv.fit(X_train_scaled, y_train)

# 结果
print("ElasticNetCV结果:")
print(f"  最佳α: {elastic_cv.alpha_:.6f}")
print(f"  最佳l1_ratio: {elastic_cv.l1_ratio_:.2f}")
print(f"  非零系数: {np.sum(np.abs(elastic_cv.coef_) > 1e-10)}")

# 测试性能
y_cv_pred = elastic_cv.predict(X_test_scaled)
print(f"  测试MSE: {mean_squared_error(y_test, y_cv_pred):.4f}")
print(f"  测试R²: {r2_score(y_test, y_cv_pred):.4f}")

In [ ]:
# ============================================================
# 分析选中的特征
# ============================================================

selected = np.where(np.abs(elastic_cv.coef_) > 1e-10)[0]

print("\nElastic Net选中的特征:")
print(f"  选中特征索引: {selected}")
print(f"  真实重要特征: [0, 1, 2, 3, 4]")

# 可视化最终系数
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.bar(range(n_features), true_coefs, color='green', alpha=0.7, label='真实')
plt.bar(range(n_features), elastic_cv.coef_, color='red', alpha=0.5, label='Elastic Net')
plt.xlabel('特征索引')
plt.ylabel('系数值')
plt.title('系数对比', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# 系数绝对值排序
importance = np.abs(elastic_cv.coef_)
sorted_idx = np.argsort(importance)[::-1]
plt.barh(range(10), importance[sorted_idx[:10]])
plt.yticks(range(10), [f'特征{i}' for i in sorted_idx[:10]])
plt.xlabel('|系数|')
plt.title('Top 10 重要特征', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 📝 总结

### 关键要点

1. ✅ **Elastic Net**结合L1和L2正则化的优点
2. ✅ `l1_ratio`控制L1和L2的比例
3. ✅ 在特征相关时比Lasso更稳定
4. ✅ 使用ElasticNetCV可同时优化α和l1_ratio

### 模型选择指南

| 场景 | 推荐 |
|------|------|
| 特征独立，需特征选择 | Lasso |
| 特征高度相关 | Ridge 或 Elastic Net |
| 不确定特征关系 | Elastic Net |
| 需要稀疏+稳定 | Elastic Net |

### 下一步学习

- 📘 Logistic回归
- 📙 学习曲线分析
- 📕 正则化在深度学习中的应用

## 📚 参考资料

- [Scikit-learn: ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)
- Zou, H., & Hastie, T. (2005). "Regularization and variable selection via the elastic net"